In [1]:
import pandas as pd
from pkg_resources import resource_filename

In [2]:
# Parameters cells

In [3]:
# Parameters
steps = [
    "preprocess_model.ipynb",
    "stations.ipynb",
    "MWOrK.ipynb",
    "plots.ipynb",
    "nbcc_stations.ipynb",
    "combine_tables.ipynb",
]
n_jobs = 6
paths = {
    "output_notebook_path": "/data/results/intermediate/notebooks/",
    "preprocessed_model_path": "/data/results/intermediate/preprocessed_netcdf/",
    "preprocessed_stations_path": "/data/results/intermediate/preprocessed_stations/",
    "output_reconstruction_path": "/data/results/netcdf/",
    "output_tables_path": "/data/results/TableC2/",
    "output_figure_path": "/data/results/figures/",
    "mask_path": "data/masks/canada_mask_rp.nc",
    "north_mask_path": "data/masks/canada_mask_north_rp.nc",
    "nbcc_loc_path": "data/station_inputs/NBCC_2020_new_coords.xlsm",
}
nbcc_median_correction = True
dvs = {
    "RL50": {
        "station_dv": "RL50 (kPa)",
        "station_path": "data/station_inputs/sl50_rl50_for_maps.csv",
        "input_model_path": "data/model_inputs/snw_rain_CanRCM4-LE_ens35_1951-2016_max_rl50_load_ensmean.nc",
        "medians": {"value": 0.4, "action": "multiply"},
        "fill_glaciers": True,
    },
    "RHann": {
        "station_dv": "mean RH (%)",
        "station_path": "data/station_inputs/rh_annual_mean_10yr_for_maps.csv",
        "input_model_path": "data/model_inputs/hurs_CanRCM4-LE_ens15_1951-2016_ensmean.nc",
        "medians": {"value": "None", "action": "None"},
        "fill_glaciers": True,
    },
    "HDD": {
        "station_dv": "HDD (degC-day)",
        "station_path": "data/station_inputs/hdd_Tmax_Tmin_allstations_v3_for_maps.csv",
        "input_model_path": "data/model_inputs/hdd_CanRCM4-LE_ens35_1951-2016_ann_ensmean.nc",
        "medians": {"value": 4700, "action": "multiply"},
        "fill_glaciers": True,
    },
    "SL50": {
        "station_dv": "SL50 (kPa)",
        "station_path": "data/station_inputs/sl50_rl50_for_maps.csv",
        "input_model_path": "data/model_inputs/snw_CanRCM4-LE_ens35_1951-2016_max_rl50_load_ensmean.nc",
        "medians": {"value": 2.4, "action": "multiply"},
        "fill_glaciers": True,
    },
    "WP10": {
        "station_dv": "WP10 (kPa)",
        "station_path": "data/station_inputs/wpress_stations_rl10_rl50_for_maps.csv",
        "input_model_path": "data/model_inputs/wpress_CanRCM4-LE_ens35_1951-2016_max_rl10_kpa_ensmean.nc",
        "medians": {"value": 0.33, "action": "multiply"},
        "fill_glaciers": True,
    },
    "WP50": {
        "station_dv": "WP50 (kPa)",
        "station_path": "data/station_inputs/wpress_stations_rl10_rl50_for_maps.csv",
        "input_model_path": "data/model_inputs/wpress_CanRCM4-LE_ens35_1951-2016_max_rl50_kpa_ensmean.nc",
        "medians": {"value": 0.42, "action": "multiply"},
        "fill_glaciers": True,
    },
    "TJan2.5": {
        "station_dv": "TJan2.5 (degC)",
        "station_path": "data/station_inputs/janT2.5p_T1.0p_allstations_v3_min8yr_for_maps.csv",
        "input_model_path": "data/model_inputs/tas_CanRCM4-LE_ens35_1951-2016_1hr_jan2.5p_ensmean.nc",
        "medians": {"value": -25, "action": "add"},
        "fill_glaciers": True,
    },
    "TJan1.0": {
        "station_dv": "TJan1.0 (degC)",
        "station_path": "data/station_inputs/janT2.5p_T1.0p_allstations_v3_min8yr_for_maps.csv",
        "input_model_path": "data/model_inputs/tas_CanRCM4-LE_ens35_1951-2016_1hr_jan1.0p_ensmean.nc",
        "medians": {"value": -27, "action": "add"},
        "fill_glaciers": True,
    },
    "Tmin": {
        "station_dv": "Tmin (degC)",
        "station_path": "data/station_inputs/hdd_Tmax_Tmin_allstations_v3_for_maps.csv",
        "input_model_path": "data/model_inputs/tas_CanRCM4-LE_ens35_1951-2016_ann_min_ensmean.nc",
        "medians": {"value": "None", "action": "None"},
        "fill_glaciers": True,
    },
    "Tmax": {
        "station_dv": "Tmax (degC)",
        "station_path": "data/station_inputs/hdd_Tmax_Tmin_allstations_v3_for_maps.csv",
        "input_model_path": "data/model_inputs/tas_CanRCM4-LE_ens35_1951-2016_ann_max_ensmean.nc",
        "medians": {"value": "None", "action": "None"},
        "fill_glaciers": True,
    },
    "TJul97.5": {
        "station_dv": "TJul2.5 (degC)",
        "station_path": "data/station_inputs/julT97.5p_allstations_v3_min8yr_for_maps.csv",
        "input_model_path": "data/model_inputs/tas_CanRCM4-LE_ens35_1951-2016_1hr_jul97.5p_ensmean.nc",
        "medians": {"value": 29, "action": "add"},
        "fill_glaciers": True,
    },
    "TwJul97.5": {
        "station_dv": "TwJul2.5 (degC)",
        "station_path": "data/station_inputs/julTwb97.5p_allstations_v3_for_maps.csv",
        "input_model_path": "data/model_inputs/twb_CanRCM4-LE_ens35_1951-2016_1hr_jul97.5p_ensmean.nc",
        "medians": {"value": 21, "action": "add"},
        "fill_glaciers": True,
    },
    "DRWP5": {
        "station_dv": "DRWP-RL5 (Pa)",
        "station_path": "data/station_inputs/drwp_rl5_for_maps.csv",
        "input_model_path": "data/model_inputs/drwp_CanRCM4-LE_ens15_1951-2016_rl5_ensmean.nc",
        "medians": {"value": 160, "action": "multiply"},
        "fill_glaciers": True,
    },
    "PAnn": {
        "station_dv": "annual_pr (mm)",
        "station_path": "data/station_inputs/pr_annual_mean_doy_MSC_25yr_for_maps.csv",
        "input_model_path": "data/model_inputs/pr_CanRCM4-LE_ens35_1951-2016_ann_sum_ensmean.nc",
        "medians": {"value": 900, "action": "multiply"},
        "fill_glaciers": True,
    },
    "RAnn": {
        "station_dv": "annual_rain (mm)",
        "station_path": "data/station_inputs/rain_annual_mean_doy_MSC_25yr_for_maps.csv",
        "input_model_path": "data/model_inputs/rain_CanRCM4-LE_ens35_1951-2016_ann_sum_ensmean.nc",
        "medians": {"value": 730, "action": "multiply"},
        "fill_glaciers": True,
    },
    "R1d50": {
        "station_dv": "1day rain RL50 (mm)",
        "station_path": "data/station_inputs/1day_rain_rl50_for_maps.csv",
        "input_model_path": "data/model_inputs/rain_CanRCM4-LE_ens35_1951-2016_max1day_rl50_gum_lm_ensmean.nc",
        "medians": {"value": 96, "action": "multiply"},
        "fill_glaciers": True,
    },
    "MI": {
        "station_dv": "moisture_index",
        "station_path": "data/station_inputs/moisture_index_for_maps.csv",
        "input_model_path": "data/model_inputs/moisture_index_CanRCM4-LE_ens15_1951-2016_ensmean.nc",
        "medians": {"value": 0.9, "action": "multiply"},
        "fill_glaciers": True,
    },
    "R15m10": {
        "station_dv": "Gum-LM RL10 (mm)",
        "station_path": "data/station_inputs/15min_rain_rl10_for_maps.csv",
        "input_model_path": "data/model_inputs/rain_CanRCM4-LE_ens15_1951-2016_max1hr_rl10_gum_lm_ensmean.nc",
        "medians": {"value": 20, "action": "multiply"},
        "fill_glaciers": True,
    },
}
output_notebook_path = "/data/results/intermediate/notebooks/"
preprocessed_model_path = "/data/results/intermediate/preprocessed_netcdf/"
preprocessed_stations_path = "/data/results/intermediate/preprocessed_stations/"
output_reconstruction_path = "/data/results/netcdf/"
output_tables_path = "/data/results/TableC2/"
output_figure_path = "/data/results/figures/"
mask_path = "data/masks/canada_mask_rp.nc"
north_mask_path = "data/masks/canada_mask_north_rp.nc"
nbcc_loc_path = "data/station_inputs/NBCC_2020_new_coords.xlsm"


In [4]:
output_tables_path = resource_filename(
    "climpyrical",
    output_tables_path
)

dflist = []
for name in dvs.keys():
    nrc_table_path = f"{output_tables_path}{name}_TableC2.csv"
    df = pd.read_csv(nrc_table_path)
    dflist.append(df)

In [5]:
# start with combining the design values
df_combined = pd.DataFrame()
df_combined = df_combined.assign(Location = df.Location, Prov = df.Prov, lon=df.lon, lat=df.lat)
df_combined["2020 Elev (m)"] = df["2020 Elev (m)"]
for i, key in enumerate(dvs.keys()):
    df_combined[dvs[key]["station_dv"]] = dflist[i][dvs[key]["station_dv"]]

In [6]:
assert not df_combined.isnull().values.any()

In [7]:
# Sort the combined keys into final format
# As they appear in original
original_columns = {
        "Location": "Location",
        "lat": "Latitude",
        "lon": "Longitude",
        "Prov": "prov",
        "2020 Elev (m)": "elevation (m)"
}
original_dvs = {
        "TJan2.5 (degC)": "JanT2.5 (degC)",
        "TJan1.0 (degC)": "JanT1.0 (degC)",
        "TJul2.5 (degC)": "JulT2.5 (degC)",
        "TwJul2.5 (degC)":"JulTw2.5 (degC)",
        "Tmin (degC)": "Tmin (degC)",
        "Tmax (degC)": "Tmax (degC)",
        "HDD (degC-day)": "HDD (degC-day)",
        "Gum-LM RL10 (mm)": "R15m10 (mm)",
        "1day rain RL50 (mm)": "R1d50 (mm)",
        "annual_rain (mm)": "AnnR (mm)",
        "annual_pr (mm)": "AnnP (mm)",
        "mean RH (%)": "RH (%)",
        "moisture_index": "MI",
        "DRWP-RL5 (Pa)": "DRWP5 (Pa)",
        "SL50 (kPa)": "SL50 (kPa)",
        "RL50 (kPa)": "RL50 (kPa)",
        "WP10 (kPa)": "WP10 (kPa)",
        "WP50 (kPa)": "WP50 (kPa)"
    }

df_final = pd.DataFrame()
df_final = df_final.assign(Location = df.Location, Prov = df.Prov, lon=df.lon, lat=df.lat)
df_final["2020 Elev (m)"] = df["2020 Elev (m)"]
df_final = df_combined.rename(columns=original_columns)
df_final_ordered = df_final.copy()

#populate new TableC2 values with DVs from the config
for key in original_dvs.keys():
    print(key)
    try:
        df_final_ordered[original_dvs[key]] = df_final[key]
        if key != original_dvs[key]:
            df_final_ordered = df_final_ordered.drop(key, axis=1)
    except KeyError:
        continue

df_final_ordered

TJan2.5 (degC)
TJan1.0 (degC)
TJul2.5 (degC)
TwJul2.5 (degC)
Tmin (degC)
Tmax (degC)
HDD (degC-day)
Gum-LM RL10 (mm)
1day rain RL50 (mm)
annual_rain (mm)
annual_pr (mm)
mean RH (%)
moisture_index
DRWP-RL5 (Pa)
SL50 (kPa)
RL50 (kPa)
WP10 (kPa)
WP50 (kPa)


,Location,prov,Longitude,Latitude,elevation (m),RL50 (kPa),HDD (degC-day),SL50 (kPa),WP10 (kPa),WP50 (kPa),...,JanT1.0 (degC),JulT2.5 (degC),JulTw2.5 (degC),R15m10 (mm),R1d50 (mm),AnnR (mm),AnnP (mm),RH (%),MI,DRWP5 (Pa)
0,100 Mile House,BC,-121.30,51.64,926,0.120,4688.071,2.121,0.165,0.213,...,-28.199,30.524,17.201,10.377,55.838,316.890,472.953,67.909,0.454,59.707
1,Abbotsford,BC,-122.33,49.05,70,0.991,2664.723,1.702,0.293,0.379,...,-8.393,30.928,21.006,11.860,127.757,1677.600,1689.866,76.774,1.520,149.393
2,Agassiz,BC,-121.76,49.24,25,0.920,2692.101,2.284,0.235,0.304,...,-10.012,31.415,21.167,9.564,152.787,1870.103,1878.778,76.712,1.219,141.841
3,Alberni,BC,-124.80,49.27,50,0.723,2869.470,1.563,0.341,0.426,...,-5.946,28.698,19.485,10.306,148.548,1614.913,1673.889,78.141,1.176,174.341
4,Ashcroft,BC,-121.28,50.72,305,0.132,4215.124,1.027,0.244,0.321,...,-23.792,34.750,18.354,9.951,38.764,267.502,360.416,65.904,0.293,77.905
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,Kugluktuk,NU,-115.10,67.83,15,0.081,9597.444,2.399,0.481,0.627,...,-41.613,23.615,15.896,9.252,49.367,132.291,219.572,78.959,0.884,100.528
676,Nottingham Island,NU,-77.93,63.12,30,0.219,9150.920,3.000,0.570,0.698,...,-36.086,17.061,11.300,6.567,54.320,190.939,404.759,78.946,0.988,201.280
677,Resolute,NU,-94.83,74.70,20,0.098,11551.366,1.532,0.542,0.692,...,-43.115,15.282,10.975,4.281,32.043,48.785,139.398,78.665,0.982,49.983
678,Resolution Island,NU,-64.87,61.31,0,0.126,8555.868,5.467,0.541,0.688,...,-34.045,17.872,12.277,7.048,61.229,218.181,540.849,76.983,0.937,223.889


In [8]:
df_final_ordered.head()

,Location,prov,Longitude,Latitude,elevation (m),RL50 (kPa),HDD (degC-day),SL50 (kPa),WP10 (kPa),WP50 (kPa),...,JanT1.0 (degC),JulT2.5 (degC),JulTw2.5 (degC),R15m10 (mm),R1d50 (mm),AnnR (mm),AnnP (mm),RH (%),MI,DRWP5 (Pa)
0,100 Mile House,BC,-121.30,51.64,926,0.120,4688.071,2.121,0.165,0.213,...,-28.199,30.524,17.201,10.377,55.838,316.890,472.953,67.909,0.454,59.707
1,Abbotsford,BC,-122.33,49.05,70,0.991,2664.723,1.702,0.293,0.379,...,-8.393,30.928,21.006,11.860,127.757,1677.600,1689.866,76.774,1.520,149.393
2,Agassiz,BC,-121.76,49.24,25,0.920,2692.101,2.284,0.235,0.304,...,-10.012,31.415,21.167,9.564,152.787,1870.103,1878.778,76.712,1.219,141.841
3,Alberni,BC,-124.80,49.27,50,0.723,2869.470,1.563,0.341,0.426,...,-5.946,28.698,19.485,10.306,148.548,1614.913,1673.889,78.141,1.176,174.341
4,Ashcroft,BC,-121.28,50.72,305,0.132,4215.124,1.027,0.244,0.321,...,-23.792,34.750,18.354,9.951,38.764,267.502,360.416,65.904,0.293,77.905


In [9]:
combined_out_path = resource_filename("climpyrical", "data/results/TableC2/combined_dv_tablec2.csv")
df_final_ordered.round(3).to_csv(combined_out_path, index=False)